In [1]:
# # creating a large table
# import sqlite3 as sql

# conn = sql.connect('test.db')
# c = conn.cursor()
# c.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, name TEXT, age INTEGER, address TEXT, salary REAL)')
# # 100000000 4.7G (4m 41s)
# for i in range(100000):
# 	c.execute('INSERT INTO test VALUES (?, ?, ?, ?, ?)', (i, 'name'+str(i), i, 'address'+str(i), i*1000))
# conn.commit()
# conn.close()



KeyboardInterrupt



In [2]:
from bs4 import BeautifulSoup
htmlFile = open("course_directory_winter.html","r",encoding="utf8")
html = htmlFile.read()


In [3]:
# parsing the html 
# dumping the data into a txt file

soup = BeautifulSoup(html, 'html.parser')
soup.prettify()
tds = soup.find_all('td')
for td in tds:
	if td.text.strip() !="\n":
		textFile.write(td.get_text(separator=",")+"\n")




In [1]:
textFile = open("course_directory_winter.txt","r",encoding="utf8")
lines = textFile.readlines()



In [2]:
# first three letters of a course code are characters and the rest are numbers
def isCourseCode(string):
	string = string.replace(" ","")
	if len(string) == 6 and string[:3].isalpha() and string[3:].isdigit():
		return True
	
	return False

In [3]:
textFile = open("course_directory_winter.txt","r",encoding="utf8")
courses = []

for line in lines:
	if isCourseCode(line.split(",")[0]):
		courses.append(line.split(",")[0])
# monsoon 329/331 course found
# winter 283/282
print(len(courses), "Course found!")
print(courses)


283 Course found!
['COM101', 'FAC331', 'MGT136', 'MKT341', 'MKT611', 'MKT631', 'FAC641', 'TOD522', 'MGT121', 'ECO340', 'FAC104', 'FAC632', 'MGT624', 'HRT571', 'FAC244', 'EFB101', 'TOD221', 'TOD526', 'EFB203', 'HRT612', 'EFB502', 'FAC124', 'COM100', 'COM102', 'EFB508', 'MGT562', 'FAC332', 'TOD524', 'MGT542', 'COM502', 'MGT533', 'MGT543', 'ECO543', 'MAT142', 'COM109', 'COM701', 'FAC217', 'ECO520', 'MGT311', 'TOD533', 'MGT536', 'HRT632', 'INS515', 'ECO620', 'MKT103', 'STA101', 'MGT508', 'EFB608', 'FAC215', 'STA100', 'ENV555', 'FAC637', 'ECO502', 'HRT561', 'EFB609', 'TOD323', 'MGT545', 'ENV333', 'FAC533', 'MKT671', 'ECO213', 'HRT512', 'DES201', 'DES103', 'ENV533', 'DES202', 'DES203', 'COM212', 'TOD326', 'TOD210', 'TOD322', 'ECO504', 'ECO204', 'EFB602', 'MKT321', 'TOD331', 'DES301', 'ECO521', 'DES232', 'ENV591', 'FAC335', 'FAC121', 'MGT341', 'FAC114', 'COM114', 'HRT542', 'FAC631', 'FAC213', 'FAC241', 'MGT112', 'ECO330', 'MKT641', 'MKT652', 'MKT653', 'MGT622', 'MGT625', 'HRT699', 'MGT627', '

In [5]:
# jsondict= []
courseArray=[]
course = ""
flag = True
for line in lines:
	line = line.replace("\t", "")
	if isCourseCode(line.split(",")[0]):
		# courseArray.append(line.split(",")[0])
		courseArray.append(course)
		course= ""
	course+=line
# print(courseArray)





In [40]:
courseArray = list(filter(lambda a: a.replace(" ","")!="",courseArray))
def detectType(s:str):
	Days = ["Mon", "Tue","Wed","Thu","Fri","Sat","Sun"]
	if s == "Section":
		return "Section"
	elif s in Days:
		return "Day"
	elif "-" in s:
		return "Date"
	elif ":" in s:
		return "Time"
	elif s == "to":
		return "To"
	elif s.isdigit():
		return "Section No."
	else:
		return "Useless"

In [41]:
import re
lines = []
Code = ''
Level = ''
Name = ''
isBiSem = ''
Credits = ''
Faculties = ''
Semester = ''
Prerequisite = ''
Description = ''
for course in courseArray:
    lines = course.split("\n")
    lines = list(filter(lambda a: a.replace(" ","")!="",    lines))

    # print(lines)
    Code = lines[0].split(",")[0]
    Level = lines[0].split(",")[1]
    Level = Level.strip(" ").strip("[").strip("]")
    Name = lines[1]
    Credits = lines[2]
    Faculties = lines[3].split(",")
    Semester = lines[4]
    if Faculties[0] == "Not added":
        continue
    if lines[5].split(",")[0] == "PREQ_OR":
        Prerequisite = lines[5].split(",")[1]
        n = 5

        
    else:
        Prerequisite = None
        n = 4
    Description = lines[n+1]
    for i in lines[n+2:]:
        i = i.strip(",")
        i = i.replace(" ",",")
        i = i.split(",")

        i = list(filter(lambda a: a!="Quarter]",i))
        i = list(filter(lambda a: a!="[First",i))
        i = list(filter(lambda a: a!="[Second",i))
        i = list(filter(lambda a: len(a )!=0,i))
        i = list(map(lambda a: a.replace("[",""),i))
        i = list(map(lambda a: a.replace("]",""),i))
        i = list(map(lambda a: a.strip(","),i))
        for j in i:
            print(f"{j} => {detectType(j)} ")
        # print(i)



    


    # print(lines[5], lines[6],12)
    # print(Code, Level, Name, Credits, Faculties,Semester, Prerequisite, Description)

View/Print => Useless 
Outline => Useless 
None => Useless 
Section => Section 
1 => Section No. 
Mon => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Wed => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Section => Section 
2 => Section No. 
Tue => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Thu => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Section => Section 
3 => Section No. 
Mon => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Wed => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Section => Section 
4 => Section No. 
Tue => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to => To 
23-04-2023 => Date 
Thu => Day 
08:00 => Time 
to => To 
09:30 => Time 
09-01-2023 => Date 
to